The first step is find open access literature reviews. We will use the Semantic Scholar API.

In [40]:
import requests
import json
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

# API key is optional, but can help with rate limiting
SEMANTIC_SCHOLAR_API_KEY = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

data_dir = "data"

In [2]:
# Get latest dataset
dataset = requests.get('https://api.semanticscholar.org/datasets/v1/release/latest').json()

{'release_id': '2024-01-24', 'README': 'Semantic Scholar Academic Graph Datasets\n\nThese datasets provide a variety of information about research papers taken from a snapshot in time of the Semantic Scholar corpus.\n\nThis site is provided by The Allen Institute for Artificial Intelligence (“AI2”) as a service to the\nresearch community. The site is covered by AI2 Terms of Use and Privacy Policy. AI2 does not claim\nownership of any materials on this site unless specifically identified. AI2 does not exercise editorial\ncontrol over the contents of this site. AI2 respects the intellectual property rights of others. If\nyou believe your copyright or trademark is being infringed by something on this site, please follow\nthe "DMCA Notice" process set out in the Terms of Use (https://allenai.org/terms).\n\nSAMPLE DATA ACCESS\nSample data files can be downloaded with the following UNIX command:\n\nfor f in $(curl https://s3-us-west-2.amazonaws.com/ai2-s2ag/samples/MANIFEST.txt)\n  do curl -

In [67]:
# Get open access reviews with a minimum citation count. The citation count serves as a proxy for quality.
# See See https://api.semanticscholar.org/api-docs/#tag/Paper-Data/operation/get_graph_paper_bulk_search
query = f"https://api.semanticscholar.org/graph/v1/paper/search/bulk?isOpenAccess&openAccessPdf&publicationTypes=Review&minCitationCount=200&fields=title,externalIds,citationCount,fieldsOfStudy,venue,year"
headers = {
    'x-api-key': SEMANTIC_SCHOLAR_API_KEY,
    'Content-Type': 'application/json'
}
response = requests.get(query, headers=headers)
data = response.json()
total = data['total']
data

{'total': 117090,
 'token': 'PCOKWVSKJJGM4TWNJNI3EUSQJIWVNUSRKBFEYK2JFUBHFI4VBQGJGUQMZSGM3DIMSOGVFDMNKPGS2LCN2MGFEDGSJTJMYTJNSKJZELJNZXGS2ERNSRRGWAPKNQKBM',
 'data': [{'paperId': '00008d6c3dc6ad96622d601c8dad4a1b05c33946',
   'externalIds': {'MAG': '2018797083',
    'DOI': '10.1063/1.3552291',
    'CorpusId': 121773812},
   'title': 'Tunable, continuous-wave Terahertz photomixer sources and applications',
   'venue': '',
   'year': 2011,
   'citationCount': 432,
   'fieldsOfStudy': ['Physics']},
  {'paperId': '0000bd1f21da8b28e58287842958ba781a051b9a',
   'externalIds': {'MAG': '1972641293',
    'DOI': '10.1182/BLOOD-2002-04-1149',
    'CorpusId': 16283567,
    'PubMed': '12351373'},
   'title': 'Current molecular models for NADPH oxidase regulation by Rac GTPase.',
   'venue': 'Blood',
   'year': 2002,
   'citationCount': 350,
   'fieldsOfStudy': ['Biology', 'Medicine']},
  {'paperId': '000153cd09823a5b65d904553b9c1d5c97054d0a',
   'externalIds': {'MAG': '2232335510',
    'DOI': '10.1

In [68]:
# Get all paper IDs in batches using the continuation token
batch_size = 1000
n_batches = total // batch_size + 1
papers = []
for i in tqdm(range(n_batches)):
    batch_q = query
    cont_token = data['token']
    if cont_token:
        batch_q += f"&token={cont_token}"
    response = requests.get(batch_q, headers=headers)
    data = response.json()
    papers.extend(data['data'])

100%|██████████| 118/118 [03:52<00:00,  1.97s/it]


In [73]:
# Assert that IDs are unique
paper_ids = [paper['paperId'] for paper in papers]
assert len(list(set(paper_ids))) == len(paper_ids) == total

In [74]:
len(papers)

117090

In [75]:
# Save paper IDs
papers_path = os.path.join(data_dir, "papers.json")
with open(papers_path, 'w') as f:
    json.dump(papers, f)

In [92]:
# Get citations for each paper
paper_details_path = os.path.join(data_dir, "paper_details.json")
details_query = 'https://api.semanticscholar.org/graph/v1/paper/batch'
batch_size = 20
n_batches = total // batch_size + 1
if os.path.exists(paper_details_path):
    with open(paper_details_path, 'r') as f:
        paper_details = json.load(f)
paper_details = {}
skip_existing = True
for i in tqdm(range(n_batches)):
    batch_q = details_query
    ids = paper_ids[i*batch_size:(i+1)*batch_size]
    if skip_existing:
        ids = [id for id in ids if id not in paper_details]
    if len(ids) == 0:
        continue
    response = requests.post(batch_q, headers=headers, json={"ids": ids}, params={"fields": "citations.title,citations.year,citations.abstract"})
    data = response.json()
    for paper in data:
        try:
            paper_details[paper['paperId']] = paper
        except TypeError:
            print(f"Error: {paper}")
            continue
    with open(paper_details_path, 'w') as f:
        json.dump(paper_details, f)

  0%|          | 5/5855 [00:44<14:42:31,  9.05s/it]

Error: error


  0%|          | 7/5855 [01:07<17:57:14, 11.05s/it]

Error: error


  0%|          | 8/5855 [01:23<20:36:20, 12.69s/it]

...

Error: error


  3%|▎         | 187/5855 [1:01:01<40:00:23, 25.41s/it]

Error: error


  3%|▎         | 187/5855 [1:01:23<31:01:01, 19.70s/it]


KeyboardInterrupt: 